# Imports

In [5]:
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

# Key PyTorch Features

## Creating a Computational Graph in PyTorch

In [4]:
def compute_z(a, b, c):
    r1 = torch.sub(a, b)
    r2 = torch.mul(r1, 2)
    z = torch.add(r2, c)
    return z

print(f'Scalar Inputs: {compute_z(torch.tensor(1), torch.tensor(2), torch.tensor(3))}')
print(f'Rank 1 Inputs: {compute_z(torch.tensor([1]), torch.Tensor([2]), torch.Tensor([3]))}')
print(f'Rank 2 Inputs: {compute_z(torch.tensor([[1]]), torch.Tensor([[2]]), torch.Tensor([[3]]))}')

Scalar Inputs: 1
Rank 1 Inputs: tensor([1.])
Rank 2 Inputs: tensor([[1.]])


## Random Weight Initialization w/ Glorot Init

In [6]:
torch.manual_seed(1)

w = torch.empty(2, 3)
nn.init.xavier_normal_(w)
print(w)

tensor([[ 0.4183,  0.1688,  0.0390],
        [ 0.3930, -0.2858, -0.1051]])


In [7]:
class MyModule(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.w1 = torch.empty(2, 3, requires_grad=True)
        nn.init.xavier_normal_(self.w1)
        
        self.w2 = torch.empty(1, 2, requires_grad=True)
        nn.init.xavier_normal_(self.w2)

## Computing Gradients via Automatic Differentiation